In [1]:
import pandas as pd
import numpy as np
from graphdatascience import GraphDataScience

from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

# 1. Import Form Responses From Google Sheets

In [2]:
def get_credentials(cred_file, scopes):
  cred = Credentials.from_service_account_file(cred_file, scopes=scopes)
  return cred

def build_sheet(cred):
  service = build('sheets', 'v4', credentials=cred)
  sheet = service.spreadsheets()
  return sheet

def read_sheets(sheet, sheet_id, range, columns=None):
  result = sheet.values().get(spreadsheetId=sheet_id, range=range).execute()
  rows = result.get('values', [])
  if columns is None and len(rows) >= 0:
    columns = rows[0]
  elif columns:
    columns = columns
  else:
    print('If sheet is empty, provide column names')
    return
  data = rows[1:] if len(rows) > 1 else []
  df = pd.DataFrame(data, columns=columns)
  return df

def write_to_sheets(sheet, sheet_id, range, values):
  data = {'values': values}
  sheet.values().update(
      spreadsheetId=sheet_id,
      range=range,
      valueInputOption="RAW",
      body=data
  ).execute()
  print('Data berhasil ditulis')

def delete_range(sheet, sheet_id, range):
  sheet.values().clear(
      spreadsheetId=sheet_id,
      range=range
  ).execute()

In [4]:
CRED_PATH = 'credentials/bw2024-96e33661eb9c.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

credentials = get_credentials(cred_file=CRED_PATH, scopes=SCOPES)
sheet = build_sheet(cred=credentials)

In [5]:
# List out Sheet IDs

# CPL Database (CPL, Sub-CPL, Quality, Indicator, Question)
CPL_DATABASE_SHEET_ID = '1LXCK0rTLnyMPfPxBtd6hCdN31eLuT8URFtVJZyQCm8w'

# Students' Responses
# PRE (July - Oct 2025)
RESPONSES_1_SHEET_ID = "1eIGG7ENSJ0nWCcpK034N42ghBsDTcsvNDkVMusivGAI"
RESPONSES_2_SHEET_ID = "1YyHndDwrw4nlnHAurdGrRoaRvRxNCR5OlkUmhXZTUq4"
RESPONSES_3_SHEET_ID = "1USNdGq348rst_pPw5sVfGnx0VUZN5VsfO73BbCmHAOY"
# POST (Dec 2025 - Jan 2026)
RESPONSES_4_SHEET_ID = "17DndmbzO-raSOe4IdoHyvgqWVuLdgxPDoimc3KQ9ios"


In [13]:
cpl_db1_df = read_sheets(sheet=sheet, sheet_id=CPL_DATABASE_SHEET_ID, range="CPL & Quality")
cpl_db2_df = read_sheets(sheet=sheet, sheet_id=CPL_DATABASE_SHEET_ID, range="Question")

cpl_db1_df = cpl_db1_df.drop(columns=['Indikator'])

cpl_db1_df = cpl_db1_df.replace('', np.nan)
cpl_db1_df = cpl_db1_df.ffill()

cpl_db2_df = cpl_db2_df.replace('', np.nan)
cpl_db2_df = cpl_db2_df.ffill()

cpl_db1_df = cpl_db1_df.merge(cpl_db2_df[['Quality', 'Kualitas']], on="Quality", how='left')

C:\Users\PC\AppData\Local\Temp\ipykernel_5824\1777038985.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cpl_db1_df = cpl_db1_df.replace('', np.nan)


In [ ]:
pre_1_df = read_sheets(sheet=sheet, sheet_id=RESPONSES_1_SHEET_ID, range="Form Responses 1")
pre_2_df =read_sheets(sheet=sheet, sheet_id=RESPONSES_2_SHEET_ID, range="Form Responses 1")
pre_3_df = read_sheets(sheet=sheet, sheet_id=RESPONSES_3_SHEET_ID, range="Form Responses 1")
post_df = read_sheets(sheet=sheet, sheet_id=RESPONSES_4_SHEET_ID, range="Form Responses 1")

demografi_df = pre_1_df

# Unpivot

pre_1_df = pd.melt(pre_1_df, id_vars=['Timestamp', 'Email'], var_name='Question', value_name='Answer')
pre_2_df = pd.melt(pre_2_df, id_vars=['Timestamp', 'Email'], var_name='Question', value_name='Answer')
pre_3_df = pd.melt(pre_3_df, id_vars=['Timestamp', 'Email'], var_name='Question', value_name='Answer')

post_df = pd.melt(post_df, id_vars=['Timestamp', 'Email'], var_name='Question', value_name='Answer')
